In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
from datetime import timedelta, datetime

In [3]:
df = pd.read_csv('../data.csv')
df.head()

,wk_strt_dt,yr_nbr,qtr_nbr,prd,wk_nbr,wk_in_yr_nbr,mdip_dm,mdip_inst,mdip_nsp,mdip_auddig,...,seas_prd_12,seas_week_40,seas_week_41,seas_week_42,seas_week_43,seas_week_44,seas_week_45,seas_week_46,seas_week_47,seas_week_48
0,2014-08-03,2014,3,7,1,27,4863885,29087520,2421933,692315,...,0,0,0,0,0,0,0,0,0,0
1,2014-08-10,2014,3,7,2,28,20887502,8345120,3984494,475810,...,0,0,0,0,0,0,0,0,0,0
2,2014-08-17,2014,3,7,3,29,11097724,17276800,1846832,784732,...,0,0,0,0,0,0,0,0,0,0
3,2014-08-24,2014,3,7,4,30,1023446,18468480,2394834,1032301,...,0,0,0,0,0,0,0,0,0,0
4,2014-08-31,2014,3,8,1,31,21109811,26659920,3312008,400456,...,0,0,0,0,0,0,0,0,0,0


In [4]:
df['wk_strt_dt'] = pd.to_datetime(df['wk_strt_dt'])

In [5]:
# Function to pull Google Search Trends

from pytrends.request import TrendReq

def interest_over_time_df(df, date_column_index, kw_list):
    
    date_column_index_string = df.columns[date_column_index]
    
    pytrends = TrendReq(hl='en-US', tz = 300)
    
    pytrends.build_payload(kw_list, 
                           cat=0, 
                           timeframe=f"{str(df.iloc[0,date_column_index])[0:10]} {str(df.iloc[-1,date_column_index])[0:10]}", 
                           geo='CA', 
                           gprop='')
    
    interest_df = pytrends.interest_over_time()
    interest_df.drop('isPartial', axis=1, inplace = True)
    interest_df.reset_index(inplace = True)
        
    out_df = pd.merge(df, 
                   interest_df.reset_index(), 
                   how = 'left', 
                   left_on=date_column_index_string, 
                   right_on='date')
    
    out_df.drop(['date', 'index'], axis = 1, inplace = True)
    
    return out_df


In [6]:
# Function to pull economic data from Statistics Canada

from stats_can import StatsCan

data_vectors_dict = {'Monthly GDP Growth Rate (Annualised)': 'v65201210', 
                     'Monthly Inflation Rate (Annualised)': 'v41690973', 
                     'Unemployment Rate':'v2062815'}

def append_monthly_unemployment_rate(df, date_row):
    sc = StatsCan()
    econ_df = sc.vectors_to_df_remote('v2062815', periods = 360)
    econ_df.columns = ['unemployment_rate']
    econ_df = econ_df.reset_index()
    
    econ_df['refPer_yr_mth'] = [item for item in zip(econ_df['refPer'].dt.year, econ_df['refPer'].dt.month)]
    
    df['date_row_yr_mth'] = [item for item in zip(df[date_row].dt.year, df[date_row].dt.month)]
    
    df_merge_orig_gdp = pd.merge(df, 
                                econ_df.iloc[:,[-2,-1]], 
                                how = 'left', 
                                left_on='date_row_yr_mth', 
                                right_on='refPer_yr_mth')
    
    return df_merge_orig_gdp


In [8]:
new_df = interest_over_time_df(df, 0, ['Electronics'])

In [9]:
new_df = append_monthly_unemployment_rate(new_df, 'wk_strt_dt')

In [10]:
new_df.drop(['date_row_yr_mth', 'refPer_yr_mth'], axis = 1, inplace = True)

In [11]:
new_df

,wk_strt_dt,yr_nbr,qtr_nbr,prd,wk_nbr,wk_in_yr_nbr,mdip_dm,mdip_inst,mdip_nsp,mdip_auddig,...,seas_week_41,seas_week_42,seas_week_43,seas_week_44,seas_week_45,seas_week_46,seas_week_47,seas_week_48,Electronics,unemployment_rate
0,2014-08-03,2014,3,7,1,27,4863885,29087520,2421933,692315,...,0,0,0,0,0,0,0,0,56,7.0
1,2014-08-10,2014,3,7,2,28,20887502,8345120,3984494,475810,...,0,0,0,0,0,0,0,0,64,7.0
2,2014-08-17,2014,3,7,3,29,11097724,17276800,1846832,784732,...,0,0,0,0,0,0,0,0,59,7.0
3,2014-08-24,2014,3,7,4,30,1023446,18468480,2394834,1032301,...,0,0,0,0,0,0,0,0,69,7.0
4,2014-08-31,2014,3,8,1,31,21109811,26659920,3312008,400456,...,0,0,0,0,0,0,0,0,63,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
204,2018-07-01,2018,2,5,5,22,13038,13828188,387,2135131,...,0,0,0,0,0,0,0,0,40,5.8
205,2018-07-08,2018,2,6,1,23,0,3714294,0,1448480,...,0,0,0,0,0,0,0,0,45,5.8
206,2018-07-15,2018,2,6,2,24,2225092,5102353,0,1307764,...,0,0,0,0,0,0,0,0,51,5.8
207,2018-07-22,2018,2,6,3,25,17544331,1478566,1891,510539,...,0,0,0,0,0,0,0,0,40,5.8


In [12]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 209 entries, 0 to 208
Data columns (total 82 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   wk_strt_dt                209 non-null    datetime64[ns]
 1   yr_nbr                    209 non-null    int64         
 2   qtr_nbr                   209 non-null    int64         
 3   prd                       209 non-null    int64         
 4   wk_nbr                    209 non-null    int64         
 5   wk_in_yr_nbr              209 non-null    int64         
 6   mdip_dm                   209 non-null    int64         
 7   mdip_inst                 209 non-null    int64         
 8   mdip_nsp                  209 non-null    int64         
 9   mdip_auddig               209 non-null    int64         
 10  mdip_audtr                209 non-null    int64         
 11  mdip_vidtr                209 non-null    int64         
 12  mdip_viddig           

In [ ]:
plt.scatter(new_df.loc[:,'unemployment_rate'], new_df.loc[:,'Electronics'],)

In [ ]:
# data_vectors_dict = {'Monthly GDP Growth Rate (Annualised)': 'v65201210', 
#                      'Monthly Inflation Rate (Annualised)': 'v41690973', 
#                      'Unemployment Rate':'v2062815'
#                      }

# def get_data(data_source):
#     sc = StatsCan()
#     df = sc.vectors_to_df_remote(data_vectors_dict[data_source], periods = 360)
    
#     if data_source != 'Unemployment Rate':
#         out_df = (df-df.shift())/df*100*12
#         out_df = out_df.iloc[1:]
#     else:
#         out_df = df
    
#     out_df.columns = [data_source]
#     return(out_df.reset_index())